In [ ]:
import pandas as pd
import numpy as np
import json

file = open('MitreData.json')

obj_list = []
dict = {}

for line in file:
    obj_list.append(eval(line))

new_list = []
for item in obj_list:
    # if(len(item['tactics'])>1):
    #     for tactic in item['tactics']:
    #         dict = item
    #         dict['tactics'] = tactic
    #         new_list.append(dict)
    # else:
    dict = item
    #dict['tactics'] = item['tactics'][0]
    new_list.append(dict)

df = pd.DataFrame(new_list)
df.head()

In [ ]:
fileD = open('DataSources.json')

objD_list = []
for line in fileD:
    objD_list.append(eval(line))
    
df_DS = pd.DataFrame(objD_list)
df_DS.head()

In [ ]:
tramp=pd.read_json('https://raw.githubusercontent.com/center-for-threat-informed-defense/tram/master/data/training/bootstrap-training-data-small.json')

In [ ]:
df_tramp=tramp['sentences']

dict_tr={'id':[],'text':[]}
index=0
for i in df_tramp:
    
    idv=i['mappings'][0]['attack_id']
   
    dict_tr['id'].append(idv)
    dict_tr['text'].append(i['text'])
    
df_tr=pd.DataFrame(dict_tr)

In [ ]:
df_comb=df.join(df_tr.set_index('id'),on='id')

In [ ]:
# df_comb.drop(df.iloc[:, 8:22], inplace = True, axis = 1)
# df_comb.drop(['subtechniqueof'])
df_comb=df_comb.fillna(df_comb.mode().iloc[0])
df_comb.head()

In [ ]:
df_comb['id'].value_counts()
tids = df_comb['id'].unique()
dict_target = {}
i = 0
for x in tids:
     dict_target[x] = i
     i = i + 1
#print(dict_target)

df_comb['num_target']=df_comb['id'].map(dict_target)
df_comb.head()

In [ ]:
df_comb['combined'] = df_comb[['detection', 'description', 'mitigations', 'procedureexamples', 'tactics','text']].apply(lambda x: ",".join(x.astype(str)), axis=1)


print(df_comb['combined'])

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(df_comb['combined'], df_comb['num_target'], test_size=0.25, random_state=50)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(2,2))
#converting traning features into numeric vector
#print(x_train)
X_train = vect.fit_transform(x_train)
#converting training labels into numeric vector
# logs_list = []
# obj = {}
# obj[520] = "Logon from a rare country compared to entire organization -DUO Authentication-Cloud Authentication SSO Single Sign-On"
# logs_list[0] = obj[0]
# df = pd.DataFrame(new_list)
X_test = vect.transform(x_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

#Training and Predicting the data
mnb = MultinomialNB(alpha =0.2)

mnb.fit(X_train,y_train)

result= mnb.predict(X_test)
# print(result)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(result,y_test)

In [ ]:
def predict_tactic(num):
    #print(dict_target)
    for key, value in dict_target.items():
        if num == value:
            return key

In [ ]:
desc = ["All forms of phishing are electronically delivered social engineering"]
desc2 = ["Suspicious Process Activity- Targeted - Malicious Start Menu Startup Modification Analytic -A2B - EDR "]
test = vect.transform(desc2)
pred= mnb.predict(test)
print(pred)

r = predict_tactic(pred)
print (r)